In [121]:
import numpy as np
import matplotlib.pyplot as plt

In [122]:
# Apparatus dimensions
d = 0.004
ID = 0.02
OD = 0.08
#lambda_c = 0.05703 Cutoff wavelength of the coaxial waveguide is 0 for TE10 mode

# Speed of light
c = 299792458

# Permittivity and permeability of air
epsilon_air = 1
mu_air = 1

The following analysis uses Modern ferrites: Vol. 1, Chapter 7.3.1.1, Pages 167-168

In [127]:
def findT (filename):
    # Read the data from the text file
    data = np.loadtxt(filename, skiprows=5)
 
    # Extract the frequency and S-parameter values from the data
    frequency = data[:, 0][100]
    s11_mag = data[:, 1][100]
    s11_phase = data[:, 2][100]
    s21_mag = data[:, 3][100]
    s21_phase = data[:, 4][100]
    s12_mag = data[:, 5][100]
    s12_phase = data[:, 6][100]
    s22_mag = data[:, 7][100]
    s22_phase = data[:, 8][100]

    # Convert to complex s-parameters (DO WE NEED 10 or 20?)
    s11 = 10 ** (s11_mag / 10) * np.exp(1j * np.deg2rad(s11_phase))
    s21 = 10 ** (s21_mag / 10) * np.exp(1j * np.deg2rad(s21_phase))
    s12 = 10 ** (s12_mag / 10) * np.exp(1j * np.deg2rad(s12_phase))
    s22 = 10 ** (s22_mag / 10) * np.exp(1j * np.deg2rad(s22_phase))
    
    # Calculate the transfer matrix
    T = np.zeros((2, 2), dtype=complex)
    T[0, 0] = -(s11 * s22 + s12 * s21)
    T[0, 1] = -s22 / s21
    T[1, 0] = s11 / s21
    T[1, 1] = 1 / s21

    return frequency,T

In [128]:
def TtoS(T):
    t11 = T[0, 0]
    t21 = T[0, 1]
    t12 = T[1, 0]
    t22 = T[1, 1]

    s11 = t12 / t22
    s21 = 1 / t22
    s12 = (t11*t22-t12*t21)/t22
    s22 = -t21 / t22
    
    return s11, s21, s12, s22

In [129]:
freq, T_air = findT('07032023_Ferrite_Meas_Current/NO_SAMPLE_0A.S2P')  # T matrix of air
freq, T_ferrite = findT('07032023_Ferrite_Meas_Current/FERRITE_SAMPLE_0A.S2P')  # T matrix of ferrite

T_DUT = np.matmul(np.linalg.inv(T_air), T_ferrite)

s11, s21, s12, s22 = TtoS(T_DUT)

# Calculate the reflection and transmission coefficients
K = (s11**2-s21**2+1)/(2*s11)
R = K+np.sqrt(K**2-1)
if np.abs(R) > 1:
    R = K-np.sqrt(K**2-1)

T = (s11+s21-R)/(1-(s11+s21)*R)

# Calculate the relative permittivity and permeability
lambda_0 = c / freq

A = -((1/(2*np.pi*d)) * np.log(1/T))**2

# Non-zero cut off wavelength
#mu_r = np.sqrt(A) * (1+R)/(1-R) * 1/np.sqrt(1/lambda_0**2-1/lambda_c**2)
#epsilon_r = (lambda_0**2 / mu_r ) * (1/lambda_c**2+ A)

# Zero cut-off wavelength
mu_r = np.sqrt(A) * (1+R)/(1-R) * 1/np.sqrt(1/lambda_0**2)
epsilon_r = (lambda_0**2 / mu_r ) * (A)

print('Relative permittivity: ', epsilon_r)
print('Relative permeability: ', mu_r)

Relative permittivity:  (-0.3519707364122914+22.000856801221016j)
Relative permeability:  (0.9334825879755239-39.136572437145425j)
